<a href="https://colab.research.google.com/github/FMekujee27/RNNs/blob/main/2_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
import xlsxwriter
import pandas as pd
import torch
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoModelForSequenceClassification, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.1 MB/s eta 0:00:00


In [ ]:
news=pd.read_excel("/content/News.xlsx")
news[["Date","Author"]]=news["Date"].str.split('|', expand=True)
news=news.drop(columns=["Author"])
news.set_index("Date",inplace=True)
news.index=pd.to_datetime(news.index,format="mixed").strftime("%d-%m-%Y")
news.head()

,Headline
Date,
21-06-2024,California Senator Revives $15 Billion Oil Div...
21-06-2024,India's Oil Imports Surge: Russia Hits Record ...
21-06-2024,Gas Flows to Sabine Pass LNG Export Plant Drop...
21-06-2024,Norway To Consider Developing Nuclear Energy
21-06-2024,India Seeks Tighter Regulatory Control Over LN...


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', force_download=True)
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def calculate_sentiment(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt",
        truncation=True
    )
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    sentiment_score = (scores[:, 1]-scores[:,2]).item()
    return sentiment_score

# Apply the function to each row in the DataFrame
news["sentiment_score"] = news["Headline"].apply(calculate_sentiment)

# Print the resulting DataFrame with sentimental scores
news.head()

,Headline,sentiment_score
Date,,
21-06-2024,California Senator Revives $15 Billion Oil Div...,-0.000079
21-06-2024,India's Oil Imports Surge: Russia Hits Record ...,0.999967
21-06-2024,Gas Flows to Sabine Pass LNG Export Plant Drop...,-0.999995
21-06-2024,Norway To Consider Developing Nuclear Energy,-0.000718
21-06-2024,India Seeks Tighter Regulatory Control Over LN...,0.999053


In [ ]:
writer = pd.ExcelWriter("Scores.xlsx", engine="xlsxwriter")
news.to_excel(writer, index=True)
writer.close()